In [ ]:
%pip install langchain_cohere -q
%pip install spacy -q
%pip install psycopg2 -q
%pip install python-dotenv -q
#ignore error

In [ ]:
# now you need to run this in a terminal window
# python -m spacy download en_core_web_md
# now restart your kernel

Standard imports for the libraires we will be using in this notebook.  Try to keep your imports in the first cell so this can this code can more easliy be converted into a python program later

In [ ]:
import boto3
import pandas as pd
import json
import time
import os
import numpy as np
import pyarrow
import traceback
from langchain.embeddings import BedrockEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
import dbconnection
import psycopg2
from psycopg2 import OperationalError
from dotenv import load_dotenv

load_dotenv()
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

#### Lets define functions that will use various embedding models so we can generate vector embeddings
Spacey

In [ ]:
def generate_spacy_vector_embedding(text):
    embedder = SpacyEmbeddings(model_name="en_core_web_md")
    query_embedding = embedder.embed_query(text)

    return(np.array(query_embedding))

Cohere

In [ ]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': [text_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


Amazon Titan

In [ ]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



In [ ]:
# Let's generate a dense vector using Amazon Titan without using a np.array as a return value
def generate_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    #Note pgvector does not want a np.array as out manual method
    return(embedding)



This is the mathmatical formula to calcuate cosine similarity between 2 vectors

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



In [ ]:
def clean_value(value):
    value_str = str(value)
    cleaned_value = ''.join(char for char in value_str if char.isalnum() or char.isspace())
    return cleaned_value

In [ ]:
def limit_string_size(x, max_chars=2048):
    # Check if the input is a string
    if isinstance(x, str):
        return x[:max_chars]
    else:
        return x

In [ ]:
def print_top_values(list_stuff: list, num_items: int) -> None:
    i=0
    for item in list_stuff:
        i=i+1
        if i>num_items:
            return None
        print(item)

In [ ]:
# clean abstract text
#df = pd.read_csv('data/latest_research_articles.csv')
#df['abstract'] = df['abstract'].apply(clean_value)

#df
dft = pd.read_pickle('data/embedded_df.pkl')

### Advanced Retrieval Techniques
#### HyDE
A technique that optimizes semantic matching requires better semantic context.  What if we generated a document from the query that better match our stored document?

In [ ]:
### Retrieval from embedded sources
#Now that we have a dataframe with embedded content of interest, we can use semantic similarity to retrieve the right data to feed to an LLM

# Given the following query let's generate context that more closely matches the embedded data
query = "What is the latest research for broken ribs in children"

#### Calling the LLM with Python
Before we embed the vector with the query let's transform the query into a fake article.  This article will likely have a larger semantic overlap than the original smaller question. Using Bedrock we will now call Anthropic Claude Sonnet to generate a fictitous article.


In [ ]:
# Generate HyDE context

def generate_hyde_response(query_phrase):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    # Each model will named parameters which will likely be different depending on the providor
    model_kwargs =  { 
        "max_tokens": 400, # This is the maximum output tokens you want the model to use
        "temperature": 1,  # Temperature controls the randomness and creativity of the generated text.
        "top_k": 250,      # Top-k parameter determines the number of highest probability next word choices the model should conside
        "top_p": 0.9,      # Top-p sampling considers the cumulative probability distribution of the next word choices and sets a probability threshold
        "stop_sequences": ["\n\nHuman"],
    }
    # LangChain tooling
    model = BedrockChat(
        client=aws_client,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )
    
    human_prompt = "Given the following question \n {query} can you please generate a paragraph of text that answers the question. Be sure to use scientific \
                    medical terminology. Please just include the paragraph in your response."
    # Uses the messaging method which is required for all Claude 3 calls
    messages = [
        ("system", "You are a helpful assistant"),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)
        # LangChain at work
        chain = prompt | model | StrOutputParser()


        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"query": query_phrase})
        end_time = time.time()  # End timing
        print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno
        print(f"Errort: {exc_type}{exc_value}{exc_traceback} on {line_number}")

In [ ]:
def calculate_zscores(cosine_scores):
    zscores = []
    # Calculate the mean of the sample points
    mean = np.mean(cosine_scores)
    # Calculate the standard deviation of the sample points
    std_deviation = np.std(cosine_scores, ddof=1)  # ddof=1 for sample standard deviation
    # Calculate the z-scores for each sample point
    z_scores = [(x - mean) / std_deviation for x in cosine_scores]

    return z_scores

In [ ]:
# Now let's take the records that are greater that 1/2 the top Z-score and send those to the LLM for an answer
def best_answer(data, question):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    model_kwargs =  { 
        "max_tokens": 2048,
        "temperature": 0.0,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman"],
    }

    model = BedrockChat(
        client=aws_client,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    human_prompt = "You are to answer the question using the data in the following information.  Do not make up your answer, only use \
                    supporting data from the article, If you don't have enough data simply respond, I don't have enough information to answer that question. \
                    given the following article data {data} can you please give a concise answer to the following question. {question}"
    messages = [
        ("system", "You are a helpful assistant that can answer quesitons based on news articles you have been given."),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)

        chain = prompt | model | StrOutputParser()

        # Chain Invoke
        
    
        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"data": data,"question": question})
        end_time = time.time()  # End timing
        #print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno

        return f"ERROR generating good answer: {exc_type}{exc_value}{exc_traceback} on {line_number}"


### Vector Database for Larger Datasets
For our first example we used a local dataframe to store the contents of our raw text and embeddings, then manually calculated similarity between embeddings.  There are other tools that are better suited for larger datasets with embeddings. They are generally called vector databases.  We will explore pgvector running on a Postgresql database engine.

In [ ]:
from requests import get

ip = get('https://api.ipify.org').content.decode('utf8')
print('My public IP address is: {}'.format(ip))


In [ ]:
# Fill in your username so yoru database entry will be unique to you
#MY_USERNAME = "dkraker@calpoly.edu"
MY_USERNAME = "YOUR USERNAME HERE"

Let's define a function that will put data into our vector DB

In [ ]:
def insert_record_into_db(table_name, record, conn):

    # Dynamically generates and executes an INSERT SQL statement for PostgreSQL, handling
    # special data types like datetime objects and arrays directly.
    
    # Args:
    # - table_name (str): The name of the table into which the record will be inserted.
    # - data (dict): A dictionary representing the record to be inserted, where keys are column names
    #                  and values can include native PostgreSQL types like datetime and arrays.
    # - conn (psycopg2.connection): A psycopg2 connection object.
    
    # Generate column names and placeholders
    columns = ', '.join(record.keys())
    placeholders = ', '.join(['%s'] * len(record))  # PostgreSQL uses %s as placeholder

    # Create the INSERT INTO statement
    sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders})'
    #print("columns=", columns)
    #print("placeholders=", placeholders)
    #print(sql)
    try:
        cur = conn.cursor()
        
        cur.execute(sql, tuple(record.values()))
        conn.commit()
        print("Record inserted successfully.")
    except psycopg2.Error as e:
        print("An error occurred:", e)


### Switch from a dataframe to a database
Just like we used a dataframe earlier, now we will use a database instead.  This will allow us to store many more records and persist them to disk so we don't just keep them in memory.  Databases are much more efficient at storing large sets of data and being able to calulate cosine similarity more efficiently.

In [ ]:
# Let's load a staff report for the city of SLO that contains 10 pages of data.  We will chunk the report into 1000 character chunks
# with a 500 character overlap.  We will insert each chunk into the DB with embeddings for similarity search.
with open("data/staff-report.txt", "r") as file:
    text = file.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,  # Adjust the chunk size as needed
    chunk_overlap=512,  # Adjust the overlap between chunks as needed
)

# Split the text into chunks
chunks = text_splitter.split_text(text)

# Now for each chunk open a connection to our DB and insert a record into our generic schema

conn = dbconnection.open_connection_to_db()
try:
    # create a dataframe with new chunk raw text
    for chunk in chunks:
        data_record = {}
        v_embed = generate_vector_embedding(chunk)
        data_record["username"] = MY_USERNAME
        data_record["textattribute1"] = chunk
        data_record["textattribute2"] = ""
        data_record["textattribute3"] = ""
        data_record["textattribute4"] = ""
        data_record["textattribute5"] = ""
        data_record["textembedding1"] = v_embed
        insert_record_into_db("rag", data_record, conn)        
        
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()

Now let's use the database as a way to find the best match. Notice the 2 different commented SQL syntax differences.  We can easily compute Euclidean distance as well

In [ ]:
def run_similarity_search_pgvector(question, embedded_text, conn):
    # Cosine similarity
    #1-(textembedding1 <=> ('{embedded_text}')) as cosine_similar  \
    # Euclidean distance
    #textembedding1 <-> ('{embedded_text}') as euclidean_distance  \
    sql = f"SELECT textattribute1, textattribute2, textattribute3, textattribute4, textattribute5, \
                1-(textembedding1 <=> ('{embedded_text}')) as cosine_similar  \
                FROM public.rag \
                WHERE username = '{MY_USERNAME}' \
                ORDER BY cosine_similar DESC \
                LIMIT 50"
    #print(sql)
    cosine_scores = []
    try:
        article_text = ""
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()

        # grab the cosine scores so we can compute Z score for narrow article selection
        # need all scores so we can calc Z
        for row in rows:
            #print(row[5])
            cosine_scores.append(row[5])
            
        z_scores = calculate_zscores(cosine_scores)
        answer = "Unknown"
        article_text = ""
        zscore_index = 0
        first_z_score = z_scores[0]
        for row in rows:
            if(first_z_score/2)<z_scores[zscore_index]:
                print(f"Using chunk with a cosine match of: {row[5]} and Z-score of: {z_scores[zscore_index]}")
                article_text = article_text + row[0] + "\n"
            zscore_index += 1
        #print(article_text)
        answer = best_answer(article_text, question)
        #Close cursor and connection
        cur.close()
        return answer
        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()
        
  

In [ ]:
#query = "What is the city recommending?"
query = "What will it cost to replace the mixer gearbox?"

query_vector = generate_vector_embedding(query)

conn = dbconnection.open_connection_to_db()

run_similarity_search_pgvector(query, query_vector, conn)

In [ ]:
def view_my_data(conn):
    try:
        cur = conn.cursor()

        # SQL statement to delete rows where username is 'bob'
        sql = f"SELECT textattribute1, textattribute2, textattribute3, textattribute4, textattribute5 FROM public.rag WHERE username = '{MY_USERNAME}'"
    
        # Execute the SQL statement
        cur.execute(sql)
        rows = cur.fetchall()

        # grab the cosine scores so we can compute Z score for narrow article selection
        for row in rows:
            print(row[0], row[1], row[2], row[3], row[4])
    
        

        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()
    

In [ ]:
conn = dbconnection.open_connection_to_db()
view_my_data(conn)

In [ ]:
def purge_my_data(conn):
    
    try:
        cur = conn.cursor()

        # SQL statement to delete rows where username is 'bob'
        sql = f"DELETE FROM public.rag WHERE username = '{MY_USERNAME}'"
    
        # Execute the SQL statement
        cur.execute(sql)
    
        # Commit the changes to the database
        conn.commit()
    
        # Get the number of affected rows
        deleted_rows = cur.rowcount
        print(f"{deleted_rows} row(s) deleted.")

        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()

In [ ]:
conn = dbconnection.open_connection_to_db()
purge_my_data(conn)